In [2]:
from data import load_data, generate_wrong_proof

train_data, test_data = load_data()

c:\Arjun\Code\ai\ai_debate\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
for item in train_data[10:20]:
    print(item['question'])
    print('-')
    print(item['answer_correct']['proof'])
    print('-')
    print(generate_wrong_proof(item['question'], item['answer_correct']['proof']))
    print('\n\n\n')

Let S be the number of people on the first hundred years’ ship.
The second hundred years’ ship had twice as many as the first, so it had 2S people.
The third hundred years’ ship had twice as many as the second, so it had 2 * 2S = 4S people.
All the ships had S + 2S + 4S = 7S = 847 people.
Thus, the ship that the monster ate in the first hundred years had S = 847 / 7 = 121 people on it.
#### 121
-
Let S be the number of people on the first hundred years' ship.
The second hundred years' ship had twice as many as the first, so it had 2S people.
The third hundred years' ship had twice as many as the second, so it had 2 * 2S = 4S people.
All the ships had S + 2S + 4S = 7S = 847 people.
Thus, the ship that the monster ate in the first hundred years had S = 847 * 7 = 5,929 people on it.
#### 5929




He saved up $110 total because 95 + 15 = 110
He saved $15 from his allowance because 3 x 5 = 15
He earned $60 mowing lawns because 4 x 15 = 60
He earned $35 shoveling driveways because 110 - 60 -